# Hospital overcapacity

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt

%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Get state hospital utilization reports

In [3]:
before = pd.read_csv(
    "../hospitals/output/hospital_utilization.csv", dtype={"fac_no": str}
)

In [4]:
before = before[before["icu_beds"] > 0].sort_values("fac_name", ascending=True)

In [5]:
state_hospital_names = before["fac_name"].to_list()

In [6]:
before.head(10)

,fac_no,fac_name,fac_city,county,license_no,fac_zip,icu_beds,tot_lic_beds
488,106580996,ADVENTIST HEALTH AND RIDEOUT,MARYSVILLE,Yuba,230000126,95901,24,221
77,106150788,ADVENTIST HEALTH BAKERSFIELD,BAKERSFIELD,Kern,120000187,93301,34,254
85,106171049,ADVENTIST HEALTH CLEARLAKE,CLEARLAKE,Lake,110000174,95422,4,32
121,106190323,ADVENTIST HEALTH GLENDALE,GLENDALE,Los Angeles,930000059,91206,42,515
84,106164029,ADVENTIST HEALTH HANFORD,HANFORD,Kings,550001439,93230,22,173
209,106234038,ADVENTIST HEALTH HOWARD MEMORIAL,WILLITS,Mendocino,110000013,95490,4,25
381,106390923,ADVENTIST HEALTH LODI MEMORIAL,LODI,San Joaquin,030000056,95240-5179,10,194
482,106560525,ADVENTIST HEALTH SIMI VALLEY,SIMI VALLEY,Ventura,050000216,93065,24,144
474,106554011,ADVENTIST HEALTH SONORA - GREENLEY,SONORA,Tuolumne,030000094,95370,6,140
221,106281078,ADVENTIST HEALTH ST. HELENA,ST. HELENA,Napa,110000073,94574,12,151


### Get latest hospital capacity report from HHS

In [7]:
current = pd.read_csv("output/current_ca.csv")

In [8]:
current = current[current["total_covid_patients"] > 0].sort_values(
    "hospital", ascending=True
)

In [9]:
current["hospital"] = current["hospital"].str.upper()

In [10]:
hhs_hospital_names = current["hospital"].to_list()

In [11]:
current.head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
187,ADVENTIST HEALTH AND RIDEOUT,Short Term,050133,2020-12-25,Yuba,115,25.0,15.0,2.0,0.92,75.0,213.0,0.35,Greater Sacramento
71,ADVENTIST HEALTH BAKERSFIELD,Short Term,050455,2020-12-25,Kern,29,28.0,17.0,0.0,1.00,128.0,222.0,0.58,San Joaquin Valley
289,ADVENTIST HEALTH CLEARLAKE,Critical Access Hospitals,051317,2020-12-25,Lake,33,4.0,1.0,0.0,1.00,2.0,18.0,0.11,Northern California
18,ADVENTIST HEALTH DELANO,Short Term,050608,2020-12-25,Kern,29,12.0,9.0,0.0,1.00,26.0,36.0,0.72,San Joaquin Valley
69,ADVENTIST HEALTH HANFORD,Short Term,050121,2020-12-25,Kings,31,22.0,9.0,2.0,0.91,91.0,155.0,0.59,San Joaquin Valley
304,ADVENTIST HEALTH HOWARD MEMORIAL,Critical Access Hospitals,051310,2020-12-25,Mendocino,45,4.0,1.0,0.0,1.00,1.0,19.0,0.05,Northern California
141,ADVENTIST HEALTH LODI MEMORIAL,Short Term,050336,2020-12-25,San Joaquin,77,15.0,8.0,2.0,0.87,49.0,108.0,0.45,San Joaquin Valley
138,ADVENTIST HEALTH SIMI VALLEY,Short Term,050236,2020-12-25,Ventura,111,24.0,14.0,8.0,0.67,59.0,130.0,0.45,Southern California
265,ADVENTIST HEALTH SONORA,Short Term,050335,2020-12-25,Tuolumne,109,6.0,1.0,3.0,0.50,8.0,45.0,0.18,San Joaquin Valley
282,ADVENTIST HEALTH ST HELENA,Short Term,050013,2020-12-25,Napa,55,16.0,2.0,7.0,0.56,6.0,42.0,0.14,Bay Area


---

In [12]:
len(before)

345

In [13]:
len(current)

314

In [14]:
hhs_merge = pd.merge(
    current, before, left_on="hospital", right_on="fac_name", how="left"
)

In [15]:
state_merge = pd.merge(
    current, before, left_on="hospital", right_on="fac_name", how="right"
)

In [16]:
len(state_merge)

345

In [17]:
len(hhs_merge)

315

In [18]:
state_merge[["hospital", "fac_name"]].to_csv("output/state_names.csv", index=False)

In [19]:
hhs_merge[["hospital", "fac_name"]].to_csv("output/hhs_names.csv", index=False)

---

In [20]:
hhs_merge["icu_beds_change"] = (
    hhs_merge["total_staffed_adult_icu_beds"] - hhs_merge["icu_beds"]
)

In [21]:
hhs_merge[hhs_merge["county_y"] == "Los Angeles"][
    [
        "hospital",
        "county_x",
        "total_staffed_adult_icu_beds",
        "icu_beds",
        "icu_beds_change",
    ]
].sort_values("icu_beds_change", ascending=False).head(10)

,hospital,county_x,total_staffed_adult_icu_beds,icu_beds,icu_beds_change
36,CENTINELA HOSPITAL MEDICAL CENTER,Los Angeles,75.0,19.0,56.0
265,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,Los Angeles,50.0,5.0,45.0
214,POMONA VALLEY HOSPITAL MEDICAL CENTER,Los Angeles,73.0,38.0,35.0
70,EMANATE HEALTH INTER-COMMUNITY HOSPITAL,Los Angeles,40.0,8.0,32.0
124,KAISER FOUNDATION HOSPITAL - WEST LA,Los Angeles,62.0,31.0,31.0
99,HUNTINGTON MEMORIAL HOSPITAL,Los Angeles,68.0,38.0,30.0
215,PROVIDENCE HOLY CROSS MEDICAL CENTER,Los Angeles,41.0,12.0,29.0
152,LAKEWOOD REGIONAL MEDICAL CENTER,Los Angeles,37.0,8.0,29.0
120,KAISER FOUNDATION HOSPITAL - SOUTH BAY,Los Angeles,46.0,20.0,26.0
172,MEMORIAL HOSPITAL OF GARDENA,Los Angeles,26.0,5.0,21.0


### In late 2019, hospitals in LA County reported this many ICU beds

In [22]:
hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["icu_beds"].sum()

1135.0

### According to latest HHS reports, they've added this many

In [23]:
hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["icu_beds_change"].sum()

482.0

### Total ICU beds now

In [24]:
hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["total_staffed_adult_icu_beds"].sum()

1617.0

### PCT increase in ICU beds? 

In [25]:
round(
    (
        (
            hhs_merge[hhs_merge["county_y"] == "Los Angeles"][
                "total_staffed_adult_icu_beds"
            ].sum()
            - hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["icu_beds"].sum()
        )
        / hhs_merge[hhs_merge["county_y"] == "Los Angeles"]["icu_beds"].sum()
        * 100
    ),
    2,
)

42.47